# Chat Testing

Here I am going to be figuring out all of the different aspects of how the chatbot will work.

In [4]:
from openai import OpenAI

In [5]:
# Load Chat API Key
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path="./.env")

# Access the OPENAI_API_KEY variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Basic Load of OpenAI

In [6]:
client = OpenAI(api_key=OPENAI_API_KEY)

try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {
              "role": "user",
              "content": "Write a haiku about recursion in programming."
          }
      ]
  )

  print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

Code calls itself back,  
Layers of logic unfold,  
Endless depth of thought.


### Differentiate between Recipe, General, or Lifestyle

In [14]:
user_question = "How can I make my life easier with the diets I need to follow?"

user_diet = "celiac diet and low fodmap diet"

rules = """
1. You are the first step in a longer chatbot which has the goal of helping users
with specific dietary needs to answer general questions about the dietary needs, provide lifestyle advice 
in the context of said dietary need, or provide recipes specific to the user's needs.
2. Your job is only to differentiate the requests, so the next request to the chatbot will be able to have
a specific format.
For example, recipes will be returned in a different format from lifestyle advice, which will be
different from general queries.
3. Your job is to differentiate between general queries, lifestyle tips, and recipe requests.
4. If someone is asking for a recipe, you will respond with only 'recipe'.
5. If someone is asking general questions about their dietary needs, you will answer with "general"
6. If someone is asking about things that will be useful general knowledge living with their dietary need, you will answer "lifestyle"
7. Anything the user asks which does not pertain their diet, a diet, or the lifestyle associated with that diet, or claryfying questions, answer "not applicable"
"""




try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": f"{rules}"},
          {"role": "assistant", "content": f"{user_diet}"},
          {
              "role": "user",
              "content": f"{user_question}"
          }
      ]
  )

  print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

lifestyle


In [2]:
not

SyntaxError: invalid syntax (1364271588.py, line 1)